# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import imageio as io

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)
from skimage.transform import resize

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 25 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
no_rows = 100   # x to be resized input
no_cols = 100   # x to be resized input
no_frames = 30  # lenght of the video frames
no_channel = 3 # number rof channels in RGB images

In [5]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,no_frames)]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size# calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),no_rows,no_cols,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = io.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image_norm=image/255 #normalization
                    resize_image=resize(image_norm,(no_rows,no_cols)) #resizing
                    batch_data[folder,idx,:,:,0] = resize_image[:,:,0]#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = resize_image[:,:,1]#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = resize_image[:,:,2]#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
            
        # write the code for the remaining data points which are left after full batches
        if(len(folder_list)%batch_size)!=0:
            batch_size1 = len(folder_list)-(batch_size*num_batches)
            batch=num_batches
            batch_data = np.zeros((batch_size1,len(img_idx),no_rows,no_cols,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size1,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size1): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size1)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = io.imread(source_path+'/'+ t[folder + (batch*batch_size1)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image_norm=image/255
                    resize_image=resize(image_norm,(no_rows,no_cols))
                    batch_data[folder,idx,:,:,0] = resize_image[:,:,0]#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = resize_image[:,:,1]#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = resize_image[:,:,2]#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size1)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels
            
          
            

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [7]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D,Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.layers.recurrent import LSTM, GRU


#write your model here

# Input
input_shape=(no_frames,no_rows,no_cols,no_channel)

model = Sequential()

model.add(Conv3D(8,kernel_size=(3,3,3),input_shape=input_shape,padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(16,kernel_size=(3,3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(32,kernel_size=(3,3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(64,kernel_size=(3,3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(Dropout(0.25))
model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

#final output layer
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
optimiser = optimizers.Adam()#write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 100, 100, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 100, 100, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 100, 100, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 50, 50, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 50, 50, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 50, 50, 16)    64        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 50, 50, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [67]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


34/34 [==============================] - 119s 3s/step - loss: 0.3887 - categorical_accuracy: 0.8572 - val_loss: 0.5775 - val_categorical_accuracy: 0.7500

Epoch 00001: saving model to model_init_2020-05-2312_37_04.913544/model-00001-0.38986-0.85370-0.57749-0.75000.h5
Epoch 2/20
34/34 [==============================] - 114s 3s/step - loss: 0.3124 - categorical_accuracy: 0.8823 - val_loss: 0.5307 - val_categorical_accuracy: 0.8200

Epoch 00002: saving model to model_init_2020-05-2312_37_04.913544/model-00002-0.32010-0.87934-0.53072-0.82000.h5
Epoch 3/20
34/34 [==============================] - 122s 4s/step - loss: 0.3230 - categorical_accuracy: 0.8852 - val_loss: 0.4949 - val_categorical_accuracy: 0.8500

Epoch 00003: saving model to model_init_2020-05-2312_37_04.913544/model-00003-0.32695-0.88235-0.49488-0.85000.h5
Epoch 4/20
34/34 [==============================] - 124s 4s/step - loss: 0.2811 - categorical_accuracy: 0.9051 - val_loss: 0.5436 - val_categorical_accuracy: 0.8200

Epoch 00

In [58]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = Source path =  ./Project_data/train ; batch size = 20
20
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


34/34 [==============================] - 121s 4s/step - loss: 1.8961 - categorical_accuracy: 0.2627 - val_loss: 1.3837 - val_categorical_accuracy: 0.4600

Epoch 00001: saving model to model_init_2020-05-2312_37_04.913544/model-00001-1.90269-0.26094-1.38369-0.46000.h5
Epoch 2/20
34/34 [==============================] - 116s 3s/step - loss: 1.4266 - categorical_accuracy: 0.3878 - val_loss: 1.3222 - val_categorical_accuracy: 0.3400

Epoch 00002: saving model to model_init_2020-05-2312_37_04.913544/model-00002-1.44193-0.37255-1.32220-0.34000.h5
Epoch 3/20
34/34 [==============================] - 108s 3s/step - loss: 1.3058 - categorical_accuracy: 0.4430 - val_loss: 1.2374 - val_categorical_accuracy: 0.3700

Epoch 00003: saving model to model_init_2020-05-2312_37_04.913544/model-00003-1.32135-0.43741-1.23745-0.37000.h5
Epoch 4/20
34/34 [==============================] - 125s 4s/step - loss: 1.2153 - categorical_accuracy: 0.4702 - val_loss: 1.4103 - val_categorical_accuracy: 0.3300

Epoch 00

In [66]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


34/34 [==============================] - 121s 4s/step - loss: 2.3413 - categorical_accuracy: 0.2561 - val_loss: 1.4331 - val_categorical_accuracy: 0.3100

Epoch 00001: saving model to model_init_2020-05-2312_37_04.913544/model-00001-2.36447-0.26244-1.43311-0.31000.h5
Epoch 2/20
34/34 [==============================] - 111s 3s/step - loss: 1.4832 - categorical_accuracy: 0.3377 - val_loss: 1.4161 - val_categorical_accuracy: 0.3300

Epoch 00002: saving model to model_init_2020-05-2312_37_04.913544/model-00002-1.47970-0.33786-1.41609-0.33000.h5
Epoch 3/20
34/34 [==============================] - 114s 3s/step - loss: 1.3974 - categorical_accuracy: 0.3878 - val_loss: 1.3516 - val_categorical_accuracy: 0.4000

Epoch 00003: saving model to model_init_2020-05-2312_37_04.913544/model-00003-1.39138-0.38914-1.35161-0.40000.h5
Epoch 4/20
34/34 [==============================] - 107s 3s/step - loss: 1.2953 - categorical_accuracy: 0.4305 - val_loss: 1.2108 - val_categorical_accuracy: 0.5000

Epoch 00

#### Conv2D + RNN model

In [12]:
#write your model here
#nb_filters = [8,16,32,64]
#nb_dense = [256, 128, 5]
nb_filters = [8,16,32,64]
nb_dense = [128,64,5]
nb_classes = 5
# Input
input_shape=(no_frames,no_rows,no_cols,no_channel)

# Define model
model_2d = Sequential()
model_2d.add(TimeDistributed(Conv2D(nb_filters[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))


model_2d.add(TimeDistributed(Conv2D(nb_filters[1], (3,3),padding='same', activation='relu')))
model_2d.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_2d.add(TimeDistributed(Conv2D(nb_filters[2], (3,3),padding='same', activation='relu')))
model_2d.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_2d.add(TimeDistributed(Conv2D(nb_filters[3], (2,2),padding='same', activation='relu')))
model_2d.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_2d.add(TimeDistributed(BatchNormalization()))
model_2d.add(Dropout(0.25))

model_2d.add(TimeDistributed(Flatten()))

model_2d.add(Dense(nb_dense[0], activation='relu'))
model_2d.add(Dropout(0.50))
model_2d.add(Dense(nb_dense[1], activation='relu'))
model_2d.add(Dropout(0.50))

## using GRU as the RNN model along with softmax as our last layer.
model_2d.add(GRU(128, return_sequences=False))
model_2d.add(Dense(nb_classes, activation='softmax')) # using Softmax as last layer

In [13]:
optimiser =Adam() #write your optimizer
model_2d.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_2d.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 30, 50, 50, 8)     224       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 50, 50, 16)    1168      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 25, 25, 16)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 25, 25, 32)    4640      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 30, 12, 12, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 12, 12, 64)    8256      
_________________________________________________________________
time_distributed_7 (TimeDist (None, 30, 6, 6, 64)      0         
__________

In [14]:
model_2d.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0,use_multiprocessing=True)

Source path =  ./Project_data/val ; batch size = 25
Source path =  ./Project_data/train ; batch size = 25


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("An

Epoch 1/20
27/27 [==============================] - 326s 12s/step - loss: 1.3897 - categorical_accuracy: 0.3739 - val_loss: 1.4891 - val_categorical_accuracy: 0.4200

Epoch 00001: saving model to model_init_2020-05-2414_49_42.357099/model-00001-1.39684-0.37104-1.48914-0.42000.h5
Epoch 2/20
27/27 [==============================] - 143s 5s/step - loss: 1.1158 - categorical_accuracy: 0.5151 - val_loss: 1.3793 - val_categorical_accuracy: 0.4900

Epoch 00002: saving model to model_init_2020-05-2414_49_42.357099/model-00002-1.11397-0.51735-1.37925-0.49000.h5
Epoch 3/20
27/27 [==============================] - 146s 5s/step - loss: 0.9693 - categorical_accuracy: 0.6053 - val_loss: 1.4154 - val_categorical_accuracy: 0.5000

Epoch 00003: saving model to model_init_2020-05-2414_49_42.357099/model-00003-0.96565-0.60784-1.41536-0.50000.h5
Epoch 4/20
27/27 [==============================] - 141s 5s/step - loss: 0.7832 - categorical_accuracy: 0.6921 - val_loss: 1.2882 - val_categorical_accuracy: 0.62